In [ ]:
!pip install mysql-connector-python

In [9]:
from datetime import datetime, timedelta
import mysql.connector


def connect_to_backend_db():
    return mysql.connector.connect(
        user='newsroom_be_ro', password='4a724ce837',
        host='proxysql-backend.service.consul.taboolasyndication.com', port=6033,
        database='trc')


def convert_to_mysql_timestamp(python_datetime):
    return '{:%Y-%m-%d %H:%M:%S}'.format(python_datetime)


def get_start_and_end_timestamps(end_time):
    absolute_start_time = datetime(year=2020, month=12, day=9, hour=20, minute=0)
    if end_time <= absolute_start_time:
        raise ValueError("Invalid end time: " + str(end_time))
    start_time = end_time - timedelta(days=1)
    if start_time < absolute_start_time:
        start_time = absolute_start_time
    return convert_to_mysql_timestamp(start_time), convert_to_mysql_timestamp(end_time)


def get_sampling_sql(start_time, end_time, max_rows):
    template = """
        select id, url, title, description from trc.videos
        where create_time between '{start_time}' and '{end_time}'
            and publisher_id in (select publisher_id from trc.publisher_config where attribute like 'nlu-text-processor:nlu-provider:TBNLU-active' and value = 'true')
        order by rand()
        limit {limit}
    """
    return template.format(start_time=start_time, end_time=end_time, limit=max_rows)


def get_load_icp_sql(item_id):
    template = """
        select icp.publisher_id, icp.item_id, icp.item_type, icp.probability, icp.algorithm, c.name
        from trc.item_clusters_prob  icp inner join trc.`clusters` c on icp.cluster_id = c.id
        where icp.item_id = {item_id}
    """
    return template.format(item_id=item_id)


def sample_items(start_time, end_time, max_rows):
    conn = None
    cur = None
    try:
        conn = connect_to_backend_db()
        cur = conn.cursor()
        cur.execute(get_sampling_sql(start_time, end_time, max_rows))
        result_rows = cur.fetchall()
        items = []
        for row in result_rows:
            item = {
                "id": row[0],
                "url": row[1],
                "title": row[2],
                "desc": row[3]
            }
            items.append(item)
    finally:
        try:
            if cur is not None:
                cur.close()
                cur = None
        finally:
            pass
        if conn is not None:
            conn.close()
            conn = None
    return items


def load_item_icp_data(conn, item_id):
    cur = None
    try:
        cur = conn.cursor()
        cur.execute(get_load_icp_sql(item_id))
        result_rows = cur.fetchall()
        nlu_data = {}
        for row in result_rows:
            publisher_id = row[0]
            item_type = row[2]
            probability = row[3]
            algorithm = row[4]
            name = row[5]
            if algorithm.startswith("TBNLU_"):
                if "TBNLU" not in nlu_data:
                    nlu_data["TBNLU"] = {}
                provider_nlu_data = nlu_data["TBNLU"]
            else:
                if "IBM" not in nlu_data:
                    nlu_data["IBM"] = {}
                provider_nlu_data = nlu_data["IBM"]
            if algorithm.endswith("ALCHEMY_TAXONOMY"):
                if "categories" not in provider_nlu_data:
                    provider_nlu_data["categories"] = []
                provider_nlu_data["categories"].append((name, probability))
            elif algorithm.endswith("PRIMARY_CONTENT_DOMAINS"):
                if name.startswith("CONCEPT-"):
                    if "pr_concepts" not in provider_nlu_data:
                        provider_nlu_data["pr_concepts"] = []
                    provider_nlu_data["pr_concepts"].append((name[len("CONCEPT-"):], probability))
                else:
                    if "pr_entities" not in provider_nlu_data:
                        provider_nlu_data["pr_entities"] = []
                    provider_nlu_data["pr_entities"].append((name, probability))
            elif algorithm.endswith("CONTENT_DOMAINS"):
                if name.startswith("CONCEPT-"):
                    if "concepts" not in provider_nlu_data:
                        provider_nlu_data["concepts"] = []
                    provider_nlu_data["concepts"].append((name[len("CONCEPT-"):], probability))
                else:
                    if "entities" not in provider_nlu_data:
                        provider_nlu_data["entities"] = []
                    provider_nlu_data["entities"].append((name, probability))
    finally:
        try:
            if cur is not None:
                cur.close()
                cur = None
        finally:
            pass
    return nlu_data


def populate_icp_data(items):
    conn = connect_to_backend_db()
    count = 0
    try:
        for item in items:
            item["nlu_data"] = load_item_icp_data(conn, item["id"])
            count += 1
            print(count)
    finally:
        conn.close()


def print_headers():
    print("id\turl\ttitle\tdesc\tibm_cat\ttb_cat\tibm_pr_entities\ttb_pr_entities\tibm_pr_concepts\ttb_pr_concepts\tibm_entities\ttb_entities\tibm_concepts\ttb_concepts")


def print_single_row(item):
    empty_value = str([])
    ibm_cat = empty_value
    ibm_pr_entities = empty_value
    ibm_pr_concepts = empty_value
    ibm_entities = empty_value
    ibm_concepts = empty_value
    tb_cat = empty_value
    tb_pr_entities = empty_value
    tb_pr_concepts = empty_value
    tb_entities = empty_value
    tb_concepts = empty_value
    
    if "nlu_data" in item:
        nlu_data = item["nlu_data"]
        if "IBM" in nlu_data:
            provider_data = nlu_data["IBM"]
            if "categories" in provider_data:
                ibm_cat = str([x[0] for x in provider_data["categories"]])
            if "pr_entities" in provider_data:
                ibm_pr_entities = str([x[0] for x in provider_data["pr_entities"]])
            if "pr_concepts" in provider_data:
                ibm_pr_concepts = str([x[0] for x in provider_data["pr_concepts"]])
            if "entities" in provider_data:
                ibm_entities = str([x[0] for x in provider_data["entities"]])
            if "concepts" in provider_data:
                ibm_concepts = str([x[0] for x in provider_data["concepts"]])
        if "TBNLU" in nlu_data:
            provider_data = nlu_data["TBNLU"]
            if "categories" in provider_data:
                tb_cat = str([x[0] for x in provider_data["categories"]])
            if "pr_entities" in provider_data:
                tb_pr_entities = str([x[0] for x in provider_data["pr_entities"]])
            if "pr_concepts" in provider_data:
                tb_pr_concepts = str([x[0] for x in provider_data["pr_concepts"]])
            if "entities" in provider_data:
                tb_entities = str([x[0] for x in provider_data["entities"]])
            if "concepts" in provider_data:
                tb_concepts = str([x[0] for x in provider_data["concepts"]])
                
    print("{id}\t{url}\t{title}\t{desc}\t{ibm_cat}\t{tb_cat}\t{ibm_pr_entities}\t{tb_pr_entities}\t{ibm_pr_concepts}\t{tb_pr_concepts}\t{ibm_entities}\t{tb_entities}\t{ibm_concepts}\t{tb_concepts}".format(
        id="'" + str(item["id"]) + "'",
        url=item["url"],
        title=item["title"],
        desc=item["desc"].decode("utf-8", "strict") if item["desc"] is not None else "",
        ibm_cat=ibm_cat,
        tb_cat=tb_cat,
        ibm_pr_entities=ibm_pr_entities,
        tb_pr_entities=tb_pr_entities,
        ibm_pr_concepts=ibm_pr_concepts,
        tb_pr_concepts=tb_pr_concepts,
        ibm_entities=ibm_entities,
        tb_entities=tb_entities,
        ibm_concepts=ibm_concepts,
        tb_concepts=tb_concepts
    ))

def print_all_rows(items):
    print_headers()
    for item in items:
        print_single_row(item)
        
def 

In [ ]:
#start_time, end_time = get_start_and_end_timestamps(datetime.now())
#sampled_items = sample_items(start_time, end_time, 1000)
populate_icp_data(sampled_items)
print_all_rows(sampled_items)

In [11]:
from datetime import datetime, timedelta
one_day_ago = datetime.now() - timedelta(days=1)
dt = datetime(year=2020, month=12, day=8, hour=19, minute=5)
dt > one_day_ago
'{:%Y-%m-%d %H:%M:%S}'.format(dt)
sampled_items[0]
# print(str([x[0] for x in sampled_items[0]["nlu_data"]["IBM"]["concepts"]]))
# ['united states senate committee on finance', 'construction', 'project', 'cuyahoga river', 'cuyahoga county ohio', 'george voinovich', 'sherrod brown', 'sewage treatment', 'lake erie', 'trail', 'prison']


{'id': -8365504447405906805,
 'url': 'https://www.kiro7.com/news/local/man-killed-shooting-kents-east-hill/FR3AUF36KNERTCT7YGMU7ZZDVE/?outputType=amp',
 'title': 'Man killed in shooting on Kent’s East Hill',
 'desc': b'A man is dead after a shooting at an apartment complex on Kent\xe2\x80\x99s East Hill early Tuesday.',
 'nlu_data': {'IBM': {'categories': [('/society/crime', 0.9520130157470703),
    ('/society/crime/personal offense/homicide', 0.9387590289115906)],
   'entities': [('Facility-mosaic hills apartments', 0.9532619714736938),
    ('GeographicFeature-east hill', 0.7271410226821899)],
   'concepts': [('apartment', 0.9488700032234192),
    ('ancient rome', 0.9150429964065552),
    ('condominium', 0.8793179988861084),
    ('south africa', 0.8600500226020813)],
   'pr_entities': [('GeographicFeature-east hill', 0.9783480167388916)],
   'pr_concepts': [('south africa', 0.8600500226020813)]},
  'TBNLU': {'categories': [('/law, govt and politics/legal issues',
     0.99199604988098